# Задание **Lite**



1. Запустите нейросеть на разном количестве тренировок, сравните результаты:

 - 10 тренировок - количество выигранных очков?
 - 20 тренировок - количество выигранных очков?
 - 30 тренировок - количество выигранных очков?

2. Проведите небольшую тренировочную сессию, изменив архитектуру нейросети:

 - изменив количество связей в Dense слое,
 - добавив Dense слой,
 - добавив Conv слой.

# Содержание:

- Установка и подключение бибилиотек
 - Основные функции
  -Предобработка данных
  - Функция потерь в соответствии с вознаграждением
  - Генерация игрового эпизода с участием нейросети
  - Эффективно определяем вознаграждение
  - Создадие серии игровых эпизодов и обучение
  - Импорт библиотек для записи и воспроизведения видео
  - Обучение на игровых эпизодах
  - Создание Pandas таблицы
- Задание 1
  - Простая нейронная сеть
    - 10 - Эпизодов в каждой тренировке
    - 20 - Эпизодов в каждой тренировке
    - 30 - Эпизодов в каждой тренировке
- Проведите небольшую тренировочную сессию, изменив архитектуру нейросети:
  - 2.1 Изменив количество связей в Dense слое
    - 2.1.1. Число нейронов в Dense слое - 50
    - 2.1.2. Число нейронов в Dense слое - 200
  - 2.2. Добавив Dense слой
  - 2.3. Добавив Conv слой
- Общая таблица
- Выводы:

**За основу будет взят ноутбук из урока с добавлением новых функций подстчета выигранных очков. А так же добавления новых архитектур неронных сетей с новыми Dense и Conv2D слоями.**

# Установка и подключение бибилиотек

In [ ]:
!pip install gym==0.18.3
!pip install atari-py==0.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657533 sha256=c1b9740e3e7945208947f15ae5bc38b8784ec4efacad561e14a48ed11a08be3a
  Stored in directory: /root/.cache/pip/wheels/1a/ec/6d/705d53925f481ab70fd48ec7728558745eeae14dfda3b49c99
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.1 MB/s 
  Attempting uninstall: atari-py
    Found existing installation: atari-py 0.2.9
    Uninstalling atari-py-0.2.9:
      Successfully uninstalled atari-py-0.2.9


In [ ]:
import gym
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda, Conv2D, MaxPooling2D, Reshape, Multiply
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import time

%matplotlib inline

In [ ]:
#Отключаем функционал из tf2.0, что бы код был совместим с современным тензорфлоуом
tf.compat.v1.disable_eager_execution()

In [ ]:
# Подключаем гугл диск для сохранения весов нейроных сетей
# from google.colab import drive
# drive.mount('/content/drive')

# Основные функции

## Предобработка данных

In [ ]:
# Функция предобработки данных перед подачей в нейросеть
def preprocessFrames(newFrame, lastFrame):

  lFrame = lastFrame.astype(np.int32)
  lFrame[lFrame==144] = 0
  lFrame[lFrame==72] = 0

  nFrame = newFrame.astype(np.int32)
  nFrame[nFrame==144] = 0
  nFrame[nFrame==72] = 0

  deltaFrame = nFrame - lFrame

  deltaFrame = deltaFrame[35:195]
  deltaFrame = deltaFrame[::2, ::2, 0]

  maxValue = deltaFrame.max() if deltaFrame.max() > abs(deltaFrame.min()) else abs(deltaFrame.min())

  if maxValue !=0:
    deltaFrame = deltaFrame / maxValue
  
  return deltaFrame

## Функция потерь в соответствии с вознаграждением

In [ ]:
# Самописная функция потерь
def rewardedLoss(episodeReward):
  def loss(yTrue, yPred):
    tmpPred = Lambda(lambda x: keras.backend.clip(x, 0.05, 0.95))(yPred)
    tmpLoss = Lambda(lambda x: -yTrue*keras.backend.log(x)-(1-yTrue)*(keras.backend.log(1-x)))(tmpPred)
    policyLoss = Multiply()([tmpLoss, episodeReward])
    return policyLoss
  return loss

## Генерация игрового эпизода с участием нейросети



In [ ]:
def generateEpisode(policyNetwork):
  statesList = []               # список состояний
  upDownActionList = []         # список движений
  rewardsList = []              # список наград
  networkOutputList = []        # вероятность 
  env = gym.make("Pong-v0")     # cоздание среды
  observation = env.reset()     # перезагрузка состояния среды
  newObservation = observation  # получение новое состояния
  done = False                  # завершение эпизода

  while done == False:
    processedNetworkInput = preprocessFrames(newFrame=newObservation, lastFrame=observation)
    statesList.append(processedNetworkInput)
    reshapedInput = np.expand_dims(processedNetworkInput, axis=0)

    upProbability = policyNetwork.predict(reshapedInput, batch_size=1)[0][0]
    networkOutputList.append(upProbability)
    actualAction = np.random.choice(a=[2,3], size=1, p=[upProbability, 1-upProbability])

    if actualAction == 2: 
      upDownActionList.append(1.0)
    else:
      upDownActionList.append(0.0)
    
    observation = newObservation
    newObservation, reward, done, info = env.step(actualAction)

    rewardsList.append(reward)

    if done:
      break
  
  env.close()
  
  return statesList, upDownActionList, rewardsList, networkOutputList

## Эффективно определяем вознаграждение



In [ ]:
# Функция, которая распределяет ненулевую награду для всех шагов в удачном/неудачном розыгрыше

def processRewards(rewardList):
  rewardDecay = 0.99
  tmpReward = 0
  rewardDecayed = np.zeros_like(rewardList, dtype=np.float32)

  for i in range(len(rewardList)-1, -1, -1):
    if rewardList[i] == 0:
      #print(f"{tmpReward}   *    {rewardDecay}   =   {tmpReward*rewardDecay}")
      tmpReward = tmpReward*rewardDecay
      rewardDecayed[i] = tmpReward
    else:
      tmpReward = rewardList[i]
      #print(tmpReward)
      rewardDecayed[i] = tmpReward
    
  rewardDecayed -= np.mean(rewardDecayed)
  rewardDecayed /= np.std(rewardDecayed)

  return rewardDecayed

## Создадие серии игровых эпизодов и обучение

In [ ]:
# Создадие серии игровых эпизодов и обучение
def generateEpisodeBatchesTraining(model, nBatches=10):
  env = gym.make('Pong-v0')
  batchStateList = []
  batchUpDownActionList = []
  batchRewardsList = []
  batchNetworkOutputList = []

  for i in range(nBatches):
    statesList, upDownActionList, rewardsList, networkOutputList = generateEpisode(model)
    batchStateList.extend(statesList)
    batchNetworkOutputList.extend(networkOutputList)
    batchUpDownActionList.extend(upDownActionList)
    batchRewardsList.extend(rewardsList)

    episodeReward = np.expand_dims(processRewards(batchRewardsList), 1)
    x = np.array(batchStateList)
    yTmp = np.array(batchUpDownActionList)
    yTrue = np.expand_dims(yTmp, 1)

    history = policyNetworkTrain.fit(x=[x, episodeReward], y=yTrue, epochs=10, verbose=0)

    batchLoss = history.history['loss'][-1]
    return batchStateList, batchUpDownActionList, batchRewardsList, batchNetworkOutputList, batchLoss


## Импорт библиотек для записи и воспроизведения видео

In [ ]:
from IPython.display import clear_output
!apt update && apt install xvfb && pip install pyvirtualdisplay 
from gym.wrappers import Monitor
import glob
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
clear_output()
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
# Функция записи и воспроизведения видео
def wrapEnv(env):
  env = Monitor(env, './video', force=True)
  return env

def showVideo():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = open(mp4, 'r+b').read()
    encoded = base64.b64encode(video) 
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    
  else:
    print("Could not find video")

In [ ]:
def playAndShowEpisode(policyNetwork):
    env = wrapEnv(gym.make('Pong-v0'))
    done = False
    observation = env.reset()
    newObservation = observation
    while done == False:
        processedNetworkInput = preprocessFrames(newFrame=newObservation,lastFrame=observation)
        reshapedInput = np.expand_dims(processedNetworkInput,axis=0)
 
        upProbability = policyNetwork.predict(reshapedInput,batch_size=1)[0][0]
        actualAction = np.random.choice(a=[2,3], size=1, p=[upProbability,1-upProbability])
        env.render()
        observation = newObservation
        newObservation, reward, done, info = env.step(actualAction)
    env.close()
    showVideo()

## Обучение на игровых эпизодах

In [ ]:
def ModelTrain(NetworkModel, episodes=10, trainingTimes = 30):
  points = []           # Cписок который будет хранить выигранные очки
  mean_points_list = [] # Cписок который будет хранить выигранные средний выигрыш
  for training in range(trainingTimes):
    startTime = time.time()
    statesList,upDownActionList,rewardsList,networkOutputList, batchLoss = generateEpisodeBatchesTraining(NetworkModel, episodes) 
    endTime = time.time()
    print("Тренировка = " + str((training)+1))
    print("Время тренировки = " + str(round(endTime - startTime))+"сек")
    print("Ошибка на тренировке = " + str(round(batchLoss, 5)))
    rr=np.array(rewardsList)
    print("Выиграли очков = "+ str(len(rr[rr>0])) + " Проиграли очков = " + str(len(rr[rr<0])))
    print("")
    points.append(len(rr[rr>0]))
    if training % 10 == 0:
      policyNetworkModel.save("policyNetworkModel.h5")
      policyNetworkModel.save("policyNetworkModel" + str(training)+".h5")
      with open('rewardsModelSimple.txt','a') as recordingRewards:
        recordingRewards.write("training = " + str(training) + 'выигранных очков = ' + str(len(rr[rr > 0])))
        recordingRewards.write("\n")
    mean_points = round(sum(points)/trainingTimes/episodes, 2)
    mean_points_list.append(mean_points)
  sum_mean_points = sum(mean_points_list)/trainingTimes/episodes
  print("Средний выигрыш: ", sum_mean_points)
  
  
  return policyNetworkModel, points, batchLoss, sum_mean_points

## Создание Pandas таблицы

In [ ]:
data = pd.DataFrame(columns= ['Выиграно очков', 'Средний выигрыш', 'Ошибка на тренировке', 'Время обучения сети(сек)'])

# Задание **1** 

Запустите нейросеть на разном количестве тренировок, сравните результаты:

 - 10 тренировок - количество выигранных очков?
 - 20 тренировок - количество выигранных очков?
 - 30 тренировок - количество выигранных очков?

## Простая нейронная сеть

In [ ]:
def createModel():
  inputs = Input(shape=(80,80))
  flattenLayer = Flatten()(inputs)
  fullConnected = Dense(200, activation='relu', use_bias=False)(flattenLayer)
  sigmoidOutput = Dense(1, activation='sigmoid', use_bias=False)(fullConnected)
  policyNetworkModel = Model(inputs, sigmoidOutput)
  episodeReward = Input(shape=(1,), name='episodeReward')

  policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=sigmoidOutput)
  policyNetworkTrain.compile(optimizer=RMSprop(learning_rate=0.0001), loss=rewardedLoss(episodeReward))
  return policyNetworkModel, policyNetworkTrain

## **10** - Эпизодов в каждой тренировке

In [ ]:
# Создаем рабочую и тренировочную модель
policyNetworkModel, policyNetworkTrain = createModel()

In [ ]:
start_time = time.time()
policyNetworkModel, points_10, batchLoss_10, mean_points_10 = ModelTrain(policyNetworkModel, 10)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 28сек
Ошибка на тренировке = -0.05683
Выиграли очков = 2 Проиграли очков = 21

Тренировка = 2
Время тренировки = 11сек
Ошибка на тренировке = -0.06723
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 15сек
Ошибка на тренировке = -0.07174
Выиграли очков = 3 Проиграли очков = 21

Тренировка = 4
Время тренировки = 12сек
Ошибка на тренировке = -0.07974
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 5
Время тренировки = 14сек
Ошибка на тренировке = -0.08231
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 6
Время тренировки = 11сек
Ошибка на тренировке = -0.14805
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 7
Время тренировки = 11сек
Ошибка на тренировке = -0.11301
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 8
Время тренировки = 13сек
Ошибка на тренировке = -0.10029
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 9
Время тренировки = 16сек
Ошибка на тренировке = -0.09366
Выиграли очков = 0 Проиг

In [ ]:
data.loc['Простая Dense(200) - 10 эпизодов'] = [sum(points_10), mean_points_10, batchLoss_10, end_time]

## **20** - Эпизодов в каждой тренировке

In [ ]:
# Создаем рабочую и тренировочную модель
policyNetworkModel, policyNetworkTrain = createModel()

In [ ]:
start_time = time.time()
policyNetworkModel, points_20, batchLoss_20, mean_points_20 = ModelTrain(policyNetworkModel, 20)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 19сек
Ошибка на тренировке = -0.06353
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 2
Время тренировки = 12сек
Ошибка на тренировке = -0.07173
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 14сек
Ошибка на тренировке = -0.08217
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 4
Время тренировки = 10сек
Ошибка на тренировке = -0.07864
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 5
Время тренировки = 10сек
Ошибка на тренировке = -0.08293
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 6
Время тренировки = 14сек
Ошибка на тренировке = -0.07399
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 7
Время тренировки = 13сек
Ошибка на тренировке = -0.08831
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 8
Время тренировки = 11сек
Ошибка на тренировке = -0.08428
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 9
Время тренировки = 13сек
Ошибка на тренировке = -0.08983
Выиграли очков = 0 Проиг

In [ ]:
data.loc['Простая Dense(200) - 20 эпизодов'] = [sum(points_20), mean_points_20, batchLoss_20, end_time]

## **30** - Эпизодов в каждой тренировке

In [ ]:
# Создаем рабочую и тренировочную модель
policyNetworkModel, policyNetworkTrain = createModel()

In [ ]:
start_time = time.time()
policyNetworkModel, points_30, batchLoss_30, mean_points_30 = ModelTrain(policyNetworkModel, 30)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 17сек
Ошибка на тренировке = -0.06735
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 2
Время тренировки = 10сек
Ошибка на тренировке = -0.06841
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 19сек
Ошибка на тренировке = -0.09541
Выиграли очков = 5 Проиграли очков = 21

Тренировка = 4
Время тренировки = 12сек
Ошибка на тренировке = -0.0699
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 5
Время тренировки = 15сек
Ошибка на тренировке = -0.10503
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 6
Время тренировки = 13сек
Ошибка на тренировке = -0.0798
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 7
Время тренировки = 13сек
Ошибка на тренировке = -0.10848
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 8
Время тренировки = 15сек
Ошибка на тренировке = -0.09261
Выиграли очков = 2 Проиграли очков = 21

Тренировка = 9
Время тренировки = 14сек
Ошибка на тренировке = -0.13043
Выиграли очков = 0 Проигра

In [ ]:
data.loc['Простая Dense(200) - 30 эпизодов'] = [sum(points_30), mean_points_30, batchLoss_20, end_time]

In [ ]:
data

,Выиграно очков,Средний выигрыш,Ошибка на тренировке,Время обучения сети(сек)
Простая Dense(200) - 10 эпизодов,26.0,0.004733,-0.167188,477.0
Простая Dense(200) - 20 эпизодов,20.0,0.000850,-0.111790,429.0
Простая Dense(200) - 30 эпизодов,30.0,0.000567,-0.111790,419.0


# 2. Проведите небольшую тренировочную сессию, изменив архитектуру нейросети:

 - изменив количество связей в Dense слое,
 - добавив Dense слой,
 - добавив Conv слой.

## 2.1 Изменив количество связей в Dense слое

### 2.1.1. Число нейронов в Dense слое - **50**

In [ ]:
def createModel_50():
  inputs = Input(shape=(80,80))
  flattenLayer = Flatten()(inputs)
  fullConnected = Dense(50, activation='relu', use_bias=False)(flattenLayer)
  sigmoidOutput = Dense(1, activation='sigmoid', use_bias=False)(fullConnected)
  policyNetworkModel = Model(inputs, sigmoidOutput)
  episodeReward = Input(shape=(1,), name='episodeReward')

  policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=sigmoidOutput)
  policyNetworkTrain.compile(optimizer=RMSprop(learning_rate=0.0001), loss=rewardedLoss(episodeReward))
  return policyNetworkModel, policyNetworkTrain

In [ ]:
policyNetworkModel, policyNetworkTrain = createModel_50()

 - Число эпизодов **30**

In [ ]:
start_time = time.time()
policyNetworkModel_50, points, batchLoss, mean_points = ModelTrain(policyNetworkModel)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 8сек
Ошибка на тренировке = -0.03231
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 2
Время тренировки = 11сек
Ошибка на тренировке = -0.03194
Выиграли очков = 2 Проиграли очков = 21

Тренировка = 3
Время тренировки = 9сек
Ошибка на тренировке = -0.03718
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 4
Время тренировки = 11сек
Ошибка на тренировке = -0.0318
Выиграли очков = 3 Проиграли очков = 21

Тренировка = 5
Время тренировки = 10сек
Ошибка на тренировке = -0.03872
Выиграли очков = 2 Проиграли очков = 21

Тренировка = 6
Время тренировки = 9сек
Ошибка на тренировке = -0.04324
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 7
Время тренировки = 8сек
Ошибка на тренировке = -0.05261
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 8
Время тренировки = 8сек
Ошибка на тренировке = -0.05599
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 9
Время тренировки = 9сек
Ошибка на тренировке = -0.04313
Выиграли очков = 1 Проиграли оч

In [ ]:
data.loc['Простая Dense(50) - 30 эпизодов'] = [sum(points), mean_points, batchLoss, end_time]

### 2.1.2. Число нейронов в Dense слое - **100**

In [ ]:
def createModel_200():
  inputs = Input(shape=(80,80))
  flattenLayer = Flatten()(inputs)
  fullConnected = Dense(100, activation='relu', use_bias=False)(flattenLayer)
  sigmoidOutput = Dense(1, activation='sigmoid', use_bias=False)(fullConnected)
  policyNetworkModel = Model(inputs, sigmoidOutput)
  episodeReward = Input(shape=(1,), name='episodeReward')

  policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=sigmoidOutput)
  policyNetworkTrain.compile(optimizer=RMSprop(learning_rate=0.0001), loss=rewardedLoss(episodeReward))
  return policyNetworkModel, policyNetworkTrain

In [ ]:
policyNetworkModel, policyNetworkTrain = createModel_200()

 - Число эпизодов **30**

In [ ]:
start_time = time.time()
policyNetworkModel_200, points, batchLoss, mean_points = ModelTrain(policyNetworkModel)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 9сек
Ошибка на тренировке = -0.03095
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 2
Время тренировки = 7сек
Ошибка на тренировке = -0.02745
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 7сек
Ошибка на тренировке = -0.03525
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 4
Время тренировки = 8сек
Ошибка на тренировке = -0.02829
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 5
Время тренировки = 7сек
Ошибка на тренировке = -0.01897
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 6
Время тренировки = 10сек
Ошибка на тренировке = -0.03658
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 7
Время тренировки = 7сек
Ошибка на тренировке = -0.03443
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 8
Время тренировки = 8сек
Ошибка на тренировке = -0.05227
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 9
Время тренировки = 10сек
Ошибка на тренировке = -0.05108
Выиграли очков = 1 Проиграли оч

In [ ]:
data.loc['Простая Dense(100) - 30 эпизодов'] = [sum(points), mean_points, batchLoss, end_time]

## 2.2. Добавив Dense слой

Будет добавлено дополнительно еще **два** **Dense** слоя.

In [ ]:
def createModel_2():
  inputs = Input(shape=(80,80))
  flattenedLayer = Flatten()(inputs)
  fullConnected = Dense(50, activation='relu', use_bias=False)(flattenedLayer)
  fullConnected = Dense(100, activation='relu', use_bias=False)(fullConnected)
  fullConnected = Dense(200, activation='relu', use_bias=False)(fullConnected)
  sigmoidOutput = Dense(1, activation='sigmoid', use_bias=False)(fullConnected)
  policyNetworkModel = Model(inputs=inputs, outputs=sigmoidOutput)

  episodeReward = Input(shape=(1,), name='episodeReward')
  policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=sigmoidOutput)
  policyNetworkTrain.compile(optimizer=RMSprop(learning_rate=0.0001), loss=rewardedLoss(episodeReward))

  return policyNetworkModel, policyNetworkTrain

In [ ]:
policyNetworkModel, policyNetworkTrain = createModel_2()

 - Число эпизодов **30**

In [ ]:
start_time = time.time()
policyNetworkModel, points, batchLoss, mean_points = ModelTrain(policyNetworkModel)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 9сек
Ошибка на тренировке = -0.0528
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 2
Время тренировки = 7сек
Ошибка на тренировке = -0.11896
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 8сек
Ошибка на тренировке = -0.08712
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 4
Время тренировки = 7сек
Ошибка на тренировке = -0.2159
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 5
Время тренировки = 7сек
Ошибка на тренировке = -0.31928
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 6
Время тренировки = 8сек
Ошибка на тренировке = -0.32725
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 7
Время тренировки = 10сек
Ошибка на тренировке = -0.35659
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 8
Время тренировки = 10сек
Ошибка на тренировке = -0.34372
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 9
Время тренировки = 9сек
Ошибка на тренировке = -0.28241
Выиграли очков = 0 Проиграли очко

In [ ]:
data.loc['Сложная Dense(3слоя) - 30 эпизодов'] = [sum(points), mean_points, batchLoss, end_time]

In [ ]:
data

,Выиграно очков,Средний выигрыш,Ошибка на тренировке,Время обучения сети(сек)
Простая Dense(200) - 10 эпизодов,26.0,0.004733,-0.167188,477.0
Простая Dense(200) - 20 эпизодов,20.0,0.000850,-0.111790,429.0
Простая Dense(200) - 30 эпизодов,16.0,0.002133,-0.065264,261.0
Простая Dense(50) - 30 эпизодов,25.0,0.005067,-0.074051,265.0
Сложная Dense(3слоя) - 30 эпизодов,12.0,0.001033,-0.167566,270.0


## 2.3. Добавив Conv слой

In [ ]:
def createModel_3():
  inputs = Input(shape=(80,80))
  reshape = Reshape((80,80,1))(inputs)
  conv2d = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', use_bias=False)(reshape)
  maxPooling = MaxPooling2D(2)(conv2d)
  flattenedLayer = Flatten()(maxPooling)
  fullConnected = Dense(100, activation='relu', use_bias=False)(flattenedLayer)
  sigmoidOutput = Dense(1, activation='sigmoid', use_bias=False)(fullConnected)
  policyNetworkModel = Model(inputs=inputs, outputs=sigmoidOutput)

  episodeReward = Input(shape=(1,), name='episodeReward')
  policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=sigmoidOutput)
  policyNetworkTrain.compile(optimizer=RMSprop(learning_rate=0.0001), loss=rewardedLoss(episodeReward))

  return policyNetworkModel, policyNetworkTrain

In [ ]:
policyNetworkModel, policyNetworkTrain = createModel_3()

 - Число эпизодов **30**

In [ ]:
start_time = time.time()
policyNetworkModel, points, batchLoss, mean_points = ModelTrain(policyNetworkModel)
end_time = round(time.time() - start_time)
print(f'Общее время обучения: {end_time} секунд')

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Тренировка = 1
Время тренировки = 238сек
Ошибка на тренировке = -0.52177
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 2
Время тренировки = 238сек
Ошибка на тренировке = -0.47527
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 3
Время тренировки = 264сек
Ошибка на тренировке = -0.33374
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 4
Время тренировки = 201сек
Ошибка на тренировке = -0.45203
Выиграли очков = 2 Проиграли очков = 21

Тренировка = 5
Время тренировки = 331сек
Ошибка на тренировке = -0.30278
Выиграли очков = 1 Проиграли очков = 21

Тренировка = 6
Время тренировки = 234сек
Ошибка на тренировке = -0.41696
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 7
Время тренировки = 198сек
Ошибка на тренировке = -0.35281
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 8
Время тренировки = 208сек
Ошибка на тренировке = -0.49354
Выиграли очков = 0 Проиграли очков = 21

Тренировка = 9
Время тренировки = 267сек
Ошибка на тренировке = -0.40389
Выиграли очков 

In [ ]:
data.loc['Сложная Dense+Conv2D - 30 эпизодов'] = [sum(points), mean_points, batchLoss, end_time]

## Общая таблица

In [ ]:
data

,Выиграно очков,Средний выигрыш,Ошибка на тренировке,Время обучения сети(сек)
Простая Dense(200) - 10 эпизодов,26.0,0.004733,-0.167188,477.0
Простая Dense(200) - 20 эпизодов,20.0,0.000850,-0.111790,429.0
Простая Dense(200) - 30 эпизодов,16.0,0.002133,-0.065264,261.0
Простая Dense(50) - 30 эпизодов,25.0,0.005067,-0.074051,265.0
Сложная Dense(3слоя) - 30 эпизодов,12.0,0.001033,-0.167566,270.0
Сложная Dense+Conv2D - 30 эпизодов,28.0,0.003933,-0.134293,8280.0


# Выводы:

1. Как видно из таблицы больше всего выигранных очков получилось получить у сети **Dense+Conv2D** (что не значительно больше простой Dense)
2. Нейронная сеть с одним **Dense** на **50** нейронов показала результат не намного хуже, а средний выигрышь даже выше.
3. Сложная сеть с **Conv2D** слоем обучалась на порядок дольше чем простые сети. 
4. Используя простые архитектуры можно добиться результатов не хуже, а с учетом времени обучения сделать большее кол-во экспериментов.  
5. Для лучший результатов требуется больше эпизодов обучения и тренировок.
Так же подбор оптимальной архитектуры нейронной сети и гиперпараметров.